## SURFACE SM ANOMALIES FROM SMAP

https://developers.google.com/earth-engine/datasets/catalog/NASA_USDA_HSL_SMAP10KM_soil_moisture

## PART I: QUERY DATA FROM GEE SERVER

In [16]:
#installing dependencies
import ee
import geemap
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib as mpl
from pandas_profiling import ProfileReport

In [17]:
#user select raw data destination folder
destination_folder = input('destination_folder: ')

destination_folder: D:\Users\aster\Desktop\dzud-types\hoof-dzud\SM\SM-repo


In [18]:
#plot wold map
Map = geemap.Map()

In [19]:
#upload shapefile of AOE
prefectures_shp = '../SM-repo/prefectures-v3.shp'
prefectures = geemap.shp_to_ee(prefectures_shp)
Map.addLayer(prefectures, {}, 'prefectures')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Input DATE, GEE Snippet and band. Can also modify data bounds and map colors. 

In [24]:
#user input start year 
#end year automatically set to most current data

print ('Climatology Summer: 2002-2021')

season1 = str('07-01')
season2 = str('08-01')

print ('START AT', season1, 'END AT', season2)

year = 2001
while 2000 < year < 2023:
    
    print('Selected year: ', year)
    start_date = str(year)+str('-')+season1
    end_date = str(year)+str('-')+season2
    year = year + 1
    
    dataset = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture')     .filterDate (start_date, end_date)     .map(lambda image: image.clip(prefectures)) 
    ssma = dataset.select('ssma')    
    ssmaVis = {
        'min': -4,
        'max': 4,
        'palette': ['0300ff', '418504', 'efff07', 'efff07', 'ff0303']    }
    Map.setCenter(106, 47, 4)
    mean_ssma = ssma.reduce(ee.Reducer.mean())
    Map.addLayer(mean_ssma, ssmaVis, 'mean_ssma-july')
    feature = prefectures
    roi = feature.geometry()
    out_dir = os.path.join(os.path.expanduser('~'), destination_folder)
    filename = os.path.join(out_dir, 'mean_ssma-july '+ str(year)+'.tif')
    image = ssma
    geemap.ee_export_image(image, filename=filename, scale=1000, region=roi)
    out_et_stats = os.path.join(out_dir, 'mean_ssma-july'+ str(year)+'-province-table.csv')  

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Allowed output formats: csv, shp, json, kml, kmz
    # Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
    geemap.zonal_statistics(image, prefectures, out_et_stats, statistics_type='MEAN', scale=1000)
    

Climatology Summer: 2002-2021
START AT 07-01 END AT 08-01
Selected year:  2001
The ee_object must be an ee.Image.
Selected year:  2002
The ee_object must be an ee.Image.
Selected year:  2003
The ee_object must be an ee.Image.
Selected year:  2004
The ee_object must be an ee.Image.
Selected year:  2005


KeyboardInterrupt: 

### CONT. TO NOTEBOOK 2. TIF to CSV

##### THE END 👍🏽